In [2]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
tf.keras.backend.clear_session()

In [5]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [6]:
subject = 'Paraíba - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [7]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_PB.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data

,Ano,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH,Paraíba - Consumo de Cimento (t)
0,2008,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,1945.158359,0.618312,0.647756,5.483335e+08,2.546644e+09,...,15.141120,948.241851,1293.296773,22.416524,19.527739,85.157286,4.537486,0.776272,0.677959,707.783000
1,2009,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,1991.871625,0.622639,0.649827,5.302764e+08,2.679945e+09,...,15.401551,948.141401,1311.220354,22.466548,19.707229,85.814573,4.395830,0.778859,0.680849,734.672000
2,2010,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,2087.894628,0.626723,0.651346,5.046425e+08,2.891930e+09,...,15.187968,948.153818,1296.766840,22.336126,19.669010,86.268192,4.409472,0.780851,0.683306,834.169000
3,2011,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,2212.198689,0.630566,0.652325,4.854622e+08,3.091988e+09,...,14.484405,948.335984,1243.078346,22.321663,19.581222,85.940292,4.376316,0.782258,0.685344,946.248000
4,2012,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,2280.624032,0.634231,0.652453,4.867661e+08,3.189506e+09,...,13.984417,948.452874,1177.378233,22.349961,19.382833,84.829550,4.240343,0.783042,0.686846,1103.437000
5,2013,3.510722e+07,1.748211e+06,7.492332,2.846532e+07,2268.296754,0.637000,0.652741,5.225848e+08,3.093872e+09,...,13.914136,948.674218,1124.843955,22.352189,19.171814,83.798501,4.142761,0.783295,0.688000,1223.444000
6,2014,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,2266.039323,0.641416,0.652552,5.676413e+08,2.779340e+09,...,13.565055,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,0.783070,0.689500,1240.232857
7,2015,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,2276.134358,0.643285,0.651015,5.179392e+08,2.561731e+09,...,13.574441,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,0.782409,0.689458,1076.248286
8,2016,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,2291.680633,0.655721,0.660706,4.524144e+08,2.516859e+09,...,13.373177,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,0.794801,0.700800,928.236857
9,2017,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,2353.643222,0.655605,0.660497,4.213689e+08,2.497710e+09,...,13.874302,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,0.794147,0.700509,786.338000


In [8]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,3.208948e+07,1.723657e+06,7.399995,2.790503e+07,1945.158359,0.618312,0.647756,5.483335e+08,2.546644e+09,1.874883e+10,...,8.076245,15.141120,948.241851,1293.296773,22.416524,19.527739,85.157286,4.537486,0.776272,0.677959
1,3.229855e+07,1.737699e+06,7.397195,2.790008e+07,1991.871625,0.622639,0.649827,5.302764e+08,2.679945e+09,2.055015e+10,...,8.105416,15.401551,948.141401,1311.220354,22.466548,19.707229,85.814573,4.395830,0.778859,0.680849
2,3.308896e+07,1.751827e+06,7.427988,2.807403e+07,2087.894628,0.626723,0.651346,5.046425e+08,2.891930e+09,2.342247e+10,...,8.069181,15.187968,948.153818,1296.766840,22.336126,19.669010,86.268192,4.409472,0.780851,0.683306
3,3.384049e+07,1.755792e+06,7.458472,2.823647e+07,2212.198689,0.630566,0.652325,4.854622e+08,3.091988e+09,2.604265e+10,...,8.055989,14.484405,948.335984,1243.078346,22.321663,19.581222,85.940292,4.376316,0.782258,0.685344
4,3.453206e+07,1.759354e+06,7.482244,2.837952e+07,2280.624032,0.634231,0.652453,4.867661e+08,3.189506e+09,2.708753e+10,...,10.160575,13.984417,948.452874,1177.378233,22.349961,19.382833,84.829550,4.240343,0.783042,0.686846
5,3.510722e+07,1.748211e+06,7.492332,2.846532e+07,2268.296754,0.637000,0.652741,5.225848e+08,3.093872e+09,2.523394e+10,...,10.474673,13.914136,948.674218,1124.843955,22.352189,19.171814,83.798501,4.142761,0.783295,0.688000
6,3.559137e+07,1.740643e+06,7.503408,2.852940e+07,2266.039323,0.641416,0.652552,5.676413e+08,2.779340e+09,1.954361e+10,...,10.857997,13.565055,948.952159,1092.104700,22.396949,19.115911,83.444066,4.091372,0.783070,0.689500
7,3.588415e+07,1.726042e+06,7.497228,2.851564e+07,2276.134358,0.643285,0.651015,5.179392e+08,2.561731e+09,1.327596e+10,...,11.087923,13.574441,949.008244,1112.063713,22.423455,19.017059,82.914233,4.121849,0.782409,0.689458
8,3.605286e+07,1.712480e+06,7.491294,2.849183e+07,2291.680633,0.655721,0.660706,4.524144e+08,2.516859e+09,1.011370e+10,...,11.207329,13.373177,948.977114,1153.430368,22.490255,18.907433,81.942887,4.149759,0.794801,0.700800
9,3.609231e+07,1.709809e+06,7.490431,2.847638e+07,2353.643222,0.655605,0.660497,4.213689e+08,2.497710e+09,8.456309e+09,...,11.081901,13.874302,948.990503,1103.061547,22.557983,18.973761,81.889312,4.064685,0.794147,0.700509


In [9]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      734.672000
1      834.169000
2      946.248000
3     1103.437000
4     1223.444000
5     1240.232857
6     1076.248286
7      928.236857
8      786.338000
9      707.565000
10     769.080000
11    1051.694000
12    1112.087000
13    1064.438000
14            NaN
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [10]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,-1.810146,-0.456113,-1.842947,-1.843442,-1.333420,-1.649508,-1.510728,1.204752,-0.990234,0.166091,...,-1.381612,1.277422,-1.184588,1.217879,-0.309664,0.928249,0.805107,1.556229,-1.490340,-1.593272
1,-1.664704,0.306639,-1.920915,-1.865820,-1.155847,-1.312007,-1.003354,0.820657,-0.546688,0.428911,...,-1.360179,1.672391,-1.481310,1.374327,0.143568,1.589532,1.219811,0.949694,-1.061748,-1.209465
2,-1.114834,1.074001,-1.063263,-1.080348,-0.790829,-0.993520,-0.631406,0.275392,0.158672,0.847995,...,-1.386802,1.348471,-1.444631,1.248168,-1.038098,1.448723,1.506013,1.008108,-0.731901,-0.883241
3,-0.592012,1.289389,-0.214200,-0.346889,-0.318305,-0.693862,-0.391424,-0.132596,0.824346,1.230291,...,-1.396494,0.281449,-0.906526,0.779540,-1.169137,1.125292,1.299130,0.866140,-0.498786,-0.612741
4,-0.110910,1.482858,0.447883,0.299042,-0.058196,-0.408085,-0.360153,-0.104860,1.148829,1.382743,...,0.149754,-0.476832,-0.561243,0.206068,-0.912742,0.394385,0.598328,0.283940,-0.368967,-0.413254
5,0.289216,0.877577,0.728876,0.686467,-0.105057,-0.192132,-0.289670,0.657046,0.830615,1.112297,...,0.380523,-0.583419,0.092593,-0.252485,-0.892562,-0.383058,-0.052194,-0.133878,-0.326982,-0.260055
6,0.626026,0.466529,1.037358,0.975815,-0.113638,0.152293,-0.335791,1.615453,-0.215960,0.282052,...,0.662153,-1.112836,0.913611,-0.538254,-0.487021,-0.589019,-0.275819,-0.353913,-0.364315,-0.060896
7,0.829705,-0.326563,0.865248,0.913694,-0.075263,0.297969,-0.712364,0.558230,-0.940033,-0.632425,...,0.831080,-1.098601,1.079279,-0.364039,-0.246874,-0.953211,-0.610107,-0.223421,-0.473776,-0.066429
8,0.947072,-1.063170,0.699968,0.806159,-0.016166,1.267810,1.661389,-0.835563,-1.089341,-1.093812,...,0.918809,-1.403838,0.987324,-0.002965,0.358358,-1.357098,-1.222961,-0.103915,1.578876,1.439506
9,0.974517,-1.208274,0.675928,0.736391,0.219375,1.258743,1.610294,-1.495939,-1.153056,-1.335633,...,0.826656,-0.643832,1.026875,-0.442616,0.971987,-1.112732,-1.256763,-0.468181,1.470481,1.400886


In [11]:
reshaped_train_input = dfToInputRNN(train_input)

In [12]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.7207382 , -1.05574508,  0.15418991,  0.25288101,
          1.69164035,  1.03669741,  0.81733036, -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
          0.84628191,  0

In [13]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0      734.672000
1      834.169000
2      946.248000
3     1103.437000
4     1223.444000
5     1240.232857
6     1076.248286
7      928.236857
8      786.338000
9      707.565000
10     769.080000
11    1051.694000
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [14]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    1051.694
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [15]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
12,0.40411,-0.962374,-0.399635,-0.193099,2.47033,0.993295,0.480333,1.108405,2.635823,-0.189959,...,0.841228,1.184615,0.106438,-2.634304,1.662202,-0.324798,-0.839488,-2.471071,0.585058,0.817053


In [16]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.40410988, -0.96237435, -0.39963523, -0.19309931,
          2.47032988,  0.99329515,  0.48033293,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
          0.84122842,  1

In [17]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    1112.087
Name: Paraíba - Consumo de Cimento (t), dtype: float64

In [18]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh')),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        batch_size=train_input.shape[0],
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [19]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['loss'][500:])
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [20]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3767564749, 2735634426, 3728632636, 711229555, 1243154313, 3208373977, 1844394361, 1468784131, 3453036488, 104987204, 3467162731, 3103109087, 2466309396, 4229039781, 632500926, 1395545830, 4186109364, 299474324, 3942997476, 3275707441, 1962323233, 1848877706, 387164775, 3055499053, 2350609900, 1746602912, 3209618235, 3828364261, 2022971064, 327402199, 2770818854, 1918305529, 2331300438, 983550413, 29002189, 3807805917, 1482495034, 1915545586, 3613467535, 429670636, 1752228227, 3317741989, 1367038196, 3662027758, 1512819648, 1055273137, 57662017, 2982681422, 1601213854, 743251513, 1723053686, 1945843371, 970019400, 3554019740, 795594262, 1169120459, 3213228346, 108204157, 3986481313, 2758270878, 2770230051, 4176934954, 1952576964, 1831030594, 4125627999, 3699341478, 3085418161, 1666877598, 1335572793, 393943847, 3163243136, 2685376823, 1949665276, 476682059, 2930966952, 3958081671, 2629323381, 4160999602, 3602127516, 908939032, 927060406, 1645071299, 2423903711, 1327441285, 789010705, 

2023-09-09 12:40:50.509770: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 12:40:50.528029: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-09 12:40:50.528219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 2748.845458984375
winner_seed: 3767564749


Step: 1 ___________________________________________
loss: 2398.724853515625
winner_seed: 2735634426


Step: 2 ___________________________________________
loss: 1769.3095703125
winner_seed: 3728632636


Step: 3 ___________________________________________
loss: 2619.011474609375


Step: 4 ___________________________________________
loss: 1463.6494140625
winner_seed: 1243154313


Step: 5 ___________________________________________
loss: 2000.3529052734375


Step: 6 ___________________________________________
loss: 3521.296875


Step: 7 ___________________________________________
loss: 2159.694091796875


Step: 8 ___________________________________________
loss: 2653.993408203125


Step: 9 ___________________________________________
loss: 1674.0491943359375


Step: 10 ___________________________________________
loss: 5116.78955078125


Step: 11 ___________________________________________
loss: 5844.96533203125


Step: 12 ___________________

In [21]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 935000.9375
Epoch 2/10000
1/1 [==============================] - 0s 4ms/step - loss: 894404.3125
Epoch 3/10000
1/1 [==============================] - 0s 4ms/step - loss: 589296.8125
Epoch 4/10000
1/1 [==============================] - 0s 4ms/step - loss: 98801.6953
Epoch 5/10000
1/1 [==============================] - 0s 3ms/step - loss: 456420.6562
Epoch 6/10000
1/1 [==============================] - 0s 4ms/step - loss: 29368.1406
Epoch 7/10000
1/1 [==============================] - 0s 4ms/step - loss: 26804.1035
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 19355.7129
Epoch 9/10000
1/1 [==============================] - 0s 4ms/step - loss: 67939.2734
Epoch 10/10000
1/1 [==============================] - 0s 3ms/step - loss: 23611.5156
Epoch 11/10000
1/1 [==============================] - 0s 4ms/step - loss: 20844.0566
Epoch 12/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 16601.9082
Epoch 98/10000
1/1 [==============================] - 0s 4ms/step - loss: 21201.2949
Epoch 99/10000
1/1 [==============================] - 0s 4ms/step - loss: 18161.7051
Epoch 100/10000
1/1 [==============================] - 0s 4ms/step - loss: 13220.8135
Epoch 101/10000
1/1 [==============================] - 0s 4ms/step - loss: 10745.8564
Epoch 102/10000
1/1 [==============================] - 0s 4ms/step - loss: 15010.9346
Epoch 103/10000
1/1 [==============================] - 0s 4ms/step - loss: 10854.2334
Epoch 104/10000
1/1 [==============================] - 0s 4ms/step - loss: 11996.4023
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 13267.9287
Epoch 106/10000
1/1 [==============================] - 0s 4ms/step - loss: 12648.4619
Epoch 107/10000
1/1 [==============================] - 0s 4ms/step - loss: 21803.8750
Epoch 108/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 9815.4150
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 7904.8086
Epoch 194/10000
1/1 [==============================] - 0s 4ms/step - loss: 14318.1406
Epoch 195/10000
1/1 [==============================] - 0s 3ms/step - loss: 13964.3672
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 26309.8984
Epoch 197/10000
1/1 [==============================] - 0s 4ms/step - loss: 26277.3516
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 14274.9795
Epoch 199/10000
1/1 [==============================] - 0s 3ms/step - loss: 23178.2656
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 20615.7129
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 20993.0859
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 12701.1689
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 9474.4336
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 11669.9619
Epoch 289/10000
1/1 [==============================] - 0s 4ms/step - loss: 18988.5215
Epoch 290/10000
1/1 [==============================] - 0s 4ms/step - loss: 9702.7373
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 9297.4492
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 25807.1777
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 3457.2891
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 12624.9443
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 16186.3174
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 14360.0908
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 11052.2256
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - loss:

1/1 [==============================] - 0s 4ms/step - loss: 10131.5078
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 12125.7217
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 14186.0859
Epoch 385/10000
1/1 [==============================] - 0s 4ms/step - loss: 16512.2109
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 4945.1724
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 16291.7627
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 8965.7314
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 10513.9482
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 8813.6963
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 10699.4541
Epoch 392/10000
1/1 [==============================] - 0s 4ms/step - loss: 7042.8286
Epoch 393/10000
1/1 [==============================] - 0s 4ms/step - loss:

...

1/1 [==============================] - 0s 4ms/step - loss: 6426.8613
Epoch 954/10000
1/1 [==============================] - 0s 4ms/step - loss: 15139.0791
Epoch 955/10000
1/1 [==============================] - 0s 4ms/step - loss: 14811.6689
Epoch 956/10000
1/1 [==============================] - 0s 4ms/step - loss: 14883.4541
Epoch 957/10000
1/1 [==============================] - 0s 4ms/step - loss: 7628.8608
Epoch 958/10000
1/1 [==============================] - 0s 5ms/step - loss: 14432.6328
Epoch 959/10000
1/1 [==============================] - 0s 4ms/step - loss: 10935.3564
Epoch 960/10000
1/1 [==============================] - 0s 4ms/step - loss: 7841.9360
Epoch 961/10000
1/1 [==============================] - 0s 4ms/step - loss: 8548.8477
Epoch 962/10000
1/1 [==============================] - 0s 4ms/step - loss: 12929.0674
Epoch 963/10000
1/1 [==============================] - 0s 4ms/step - loss: 7449.9336
Epoch 964/10000
1/1 [==============================] - 0s 4ms/step - loss: 

1/1 [==============================] - 0s 4ms/step - loss: 19612.6992
Epoch 1049/10000
1/1 [==============================] - 0s 4ms/step - loss: 8484.3428
Epoch 1050/10000
1/1 [==============================] - 0s 3ms/step - loss: 13529.7451
Epoch 1051/10000
1/1 [==============================] - 0s 4ms/step - loss: 10876.1084
Epoch 1052/10000
1/1 [==============================] - 0s 4ms/step - loss: 17054.3262
Epoch 1053/10000
1/1 [==============================] - 0s 4ms/step - loss: 13072.5205
Epoch 1054/10000
1/1 [==============================] - 0s 4ms/step - loss: 10171.4775
Epoch 1055/10000
1/1 [==============================] - 0s 3ms/step - loss: 15017.6357
Epoch 1056/10000
1/1 [==============================] - 0s 4ms/step - loss: 12419.2080
Epoch 1057/10000
1/1 [==============================] - 0s 4ms/step - loss: 11859.2725
Epoch 1058/10000
1/1 [==============================] - 0s 3ms/step - loss: 5346.7573
Epoch 1059/10000
1/1 [==============================] - 0s 4ms

1/1 [==============================] - 0s 4ms/step - loss: 13636.0273
Epoch 1143/10000
1/1 [==============================] - 0s 4ms/step - loss: 12307.0938
Epoch 1144/10000
1/1 [==============================] - 0s 4ms/step - loss: 6686.3906
Epoch 1145/10000
1/1 [==============================] - 0s 4ms/step - loss: 7859.3403
Epoch 1146/10000
1/1 [==============================] - 0s 4ms/step - loss: 12165.4502
Epoch 1147/10000
1/1 [==============================] - 0s 5ms/step - loss: 5271.8262
Epoch 1148/10000
1/1 [==============================] - 0s 4ms/step - loss: 9605.4014
Epoch 1149/10000
1/1 [==============================] - 0s 5ms/step - loss: 14278.2188
Epoch 1150/10000
1/1 [==============================] - 0s 5ms/step - loss: 19029.0801
Epoch 1151/10000
1/1 [==============================] - 0s 5ms/step - loss: 9049.6836
Epoch 1152/10000
1/1 [==============================] - 0s 4ms/step - loss: 7792.9429
Epoch 1153/10000
1/1 [==============================] - 0s 6ms/ste

1/1 [==============================] - 0s 5ms/step - loss: 9184.9844
Epoch 1237/10000
1/1 [==============================] - 0s 4ms/step - loss: 12050.0186
Epoch 1238/10000
1/1 [==============================] - 0s 4ms/step - loss: 11012.1885
Epoch 1239/10000
1/1 [==============================] - 0s 4ms/step - loss: 5841.7886
Epoch 1240/10000
1/1 [==============================] - 0s 4ms/step - loss: 10024.2627
Epoch 1241/10000
1/1 [==============================] - 0s 4ms/step - loss: 19177.3535
Epoch 1242/10000
1/1 [==============================] - 0s 5ms/step - loss: 13902.8047
Epoch 1243/10000
1/1 [==============================] - 0s 4ms/step - loss: 7198.2534
Epoch 1244/10000
1/1 [==============================] - 0s 4ms/step - loss: 9457.4512
Epoch 1245/10000
1/1 [==============================] - 0s 4ms/step - loss: 5090.4653
Epoch 1246/10000
1/1 [==============================] - 0s 4ms/step - loss: 12712.3906
Epoch 1247/10000
1/1 [==============================] - 0s 4ms/st

Epoch 1331/10000
1/1 [==============================] - 0s 4ms/step - loss: 8961.4932
Epoch 1332/10000
1/1 [==============================] - 0s 4ms/step - loss: 10055.6104
Epoch 1333/10000
1/1 [==============================] - 0s 4ms/step - loss: 18789.7715
Epoch 1334/10000
1/1 [==============================] - 0s 4ms/step - loss: 14304.9756
Epoch 1335/10000
1/1 [==============================] - 0s 4ms/step - loss: 4694.6992
Epoch 1336/10000
1/1 [==============================] - 0s 4ms/step - loss: 8859.1436
Epoch 1337/10000
1/1 [==============================] - 0s 4ms/step - loss: 8702.7803
Epoch 1338/10000
1/1 [==============================] - 0s 4ms/step - loss: 10726.3008
Epoch 1339/10000
1/1 [==============================] - 0s 4ms/step - loss: 10267.0596
Epoch 1340/10000
1/1 [==============================] - 0s 4ms/step - loss: 10029.8936
Epoch 1341/10000
1/1 [==============================] - 0s 4ms/step - loss: 6050.2632
Epoch 1342/10000
1/1 [==========================

In [22]:
prediction = trained_model.predict(reshaped_test_input)[0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 583ms/step
[1112.087](test_target) - [967.02075](prediction) = 145.066248046875


In [23]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.13044505335182857

In [24]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [25]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
0,-0.935924,-1.223513,-0.604041,-0.676239,-1.062778,-1.236405,-1.282559,1.148205,-1.122043,-1.121418,...,-0.469856,-0.903616,1.405096,-0.918766,0.188460,-1.385026,-1.292372,1.408816,-1.274302,-1.256477
1,-0.450207,-0.002459,-0.805386,-0.737532,-0.276619,0.023664,0.125255,0.140893,-0.184474,-0.185492,...,1.390102,1.393938,-0.841397,1.390459,1.119591,0.940056,0.148861,-0.811314,0.106005,0.066144
2,1.386131,1.225973,1.409428,1.413771,1.339397,1.212741,1.157304,-1.289098,1.306518,1.306910,...,-0.920245,-0.490322,-0.563699,-0.471692,-1.308051,0.444970,1.143510,-0.597501,1.168297,1.190332


train_target:


,Paraíba - Consumo de Cimento (t)
0,734.672
1,834.169
2,946.248


test_input:


...

,Paraíba - PIB - Estadual,Paraíba - PIB - Construção Civil,Paraíba - PIB - Per Capita,Paraíba - PIB - Preços de Mercado,Paraíba - Produção de Cimento (t),Paraíba - IDH Educacao,Paraíba - IDH Renda,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Paraíba - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Paraíba - IDH Longevidade,Paraíba - IDH
12,0.40411,-0.962374,-0.399635,-0.193099,2.47033,0.993295,0.480333,1.108405,2.635823,-0.189959,...,0.841228,1.184615,0.106438,-2.634304,1.662202,-0.324798,-0.839488,-2.471071,0.585058,0.817053


test_target:


,Paraíba - Consumo de Cimento (t)
12,1112.087


1/1 [==============================] - 1s 556ms/step
Error: 145.066248046875




[931.86962890625,
 1015.2869873046875,
 1211.8006591796875,
 1238.1474609375,
 1060.160400390625,
 932.0122680664062,
 852.739501953125,
 758.5265502929688,
 758.7449340820312,
 967.020751953125]

In [26]:
display(targets)
display(predictions)

[1103.437,
 1223.444,
 1240.2328571428573,
 1076.248285714286,
 928.2368571428572,
 786.338,
 707.565,
 769.08,
 1051.694,
 1112.087]

[931.86962890625,
 1015.2869873046875,
 1211.8006591796875,
 1238.1474609375,
 1060.160400390625,
 932.0122680664062,
 852.739501953125,
 758.5265502929688,
 758.7449340820312,
 967.020751953125]

In [27]:
mae = mean_absolute_error(predictions, targets)
mae

144.139683391462

In [28]:
porcentage = mae/np.mean(targets)
porcentage

0.14416328292087616